<h2 align='center'>  Clasificador Naive Bayes </h2> 

## Teorema de Bayes

La fórmula del Teorema de Bayes es la siguiente:

$$P(H/E) = \frac{P(E/H)P(H)}{P(E)}$$

donde:

* $P(H|E)$ es la probabilidad de la hipótesis $H$ dado el evento $E$, una probabilidad posterior.


* $P(E|H)$ es la probabilidad del evento $E$ dado que la hipótesis $H$ es verdad.


* $P(H)$ es la probabilidad de la hipótesis $H$ siendo cierto (independientemente de cualquier evento relacionado), o probabilidad previa de $H$.


* $P(E)$ es la probabilidad de que ocurra el evento (independientemente de la hipótesis).

En caso de que se tenga una sola característica, el clasificador Naive Bayes calcula la probabilidad de un evento en los siguientes pasos:

**Paso 1:** Calcular la probabilidad previa para las etiquetas de clase dadas.

**Paso 2:** Determinar la probabilidad de cada atributo para cada clase.

**Paso 3:** Poner estos valores en el teorema de Bayes y calcular la probabilidad posterior.

**Paso 4:** Ver qué clase tiene una probabilidad más alta, dado que la variable de entrada pertenece a la clase de probabilidad más alta.

# Juagamos con la BDD

### Librerias

In [2]:
import pandas as pd
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

### Cargamos Dataset

In [15]:
datasetGrande = pd.read_excel('../bdd/bdd_acv1.xlsx')
columnas = ['PCR','INR', 'TTPK', 'Tiempo protombina', 'Functional outcome numérico'] # Columnas a trabajar
dataset = datasetGrande[[*columnas]]
dataset.head(7)

,PCR,INR,TTPK,Tiempo protombina,Functional outcome numérico
0,16.00,1.30,28.0,15.0,0
1,4.10,1.08,26.9,11.9,0
2,10.10,0.97,27.1,10.8,0
3,6.70,1.23,34.4,12.5,0
4,0.54,0.96,29.4,9.8,0
5,5.20,1.10,29.4,8.4,0
6,30.00,1.30,32.8,13.2,0


## Separamos los datos de entrenamiento y validación

In [10]:
X_train, X_test, y_train, y_test = train_test_split(dataset.PCR, dataset.INR, test_size = 0.3)

## Creamos el modelo

In [6]:
modelo = GaussianNB()

## Entrenamos el modelo

In [8]:
modelo.fit(X_train, y_train)

ValueError: Unknown label type: (array([0.89, 0.9 , 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.  , 1.01,
       1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1 , 1.11, 1.12,
       1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 , 1.23, 1.24, 1.26,
       1.27, 1.28, 1.29, 1.3 , 1.34, 1.35, 1.76, 2.19, 2.5 , 3.08, 3.15,
       3.54, 4.14]),)

## Predicciones

In [11]:
y_pred = modelo.predict(X_test)

ValueError: Expected 2D array, got 1D array instead:
array=[  8.7   17.43   1.    12.11   8.    10.1   10.5   35.37  11.9   14.5
   3.9   30.    10.7    1.     1.     2.54  30.     1.69   2.86   8.
   0.66  11.6   46.    16.     1.4    1.   140.74  41.4   17.    27.6
   1.04   5.3    1.6    4.9  114.     6.7    2.4    5.2    7.     2.8
   4.    16.    16.     0.87   6.4   11.9  109.91   0.54  44.96].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Determinamos la precisión del modelo

In [12]:
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

NameError: name 'y_pred' is not defined

## Verifico la matriz de Confusión

In [14]:
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

NameError: name 'y_pred' is not defined